In [2]:
#import pacakges 
import matplotlib
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt
from PIL import Image
import math
import numpy
import cv2
from pylab import *
from numpy import *
from numpy import array
import random;
from scipy.interpolate import RectBivariateSpline

# 1) Use ginput to find correspondances

In [38]:
def getPoints(image):
    img1 = Image.open(image) 
    plt.figure()
    plt.imshow(img1)
    print("Please click") 
    x = plt.ginput(4)
    return x

In [39]:
pts_src = getPoints('./Images/image1.jpg')
print(pts_src)
pts_dst=getPoints('./Images/image2.jpg')
print(pts_dst)

Please click
[(886.835830919837, 325.3765775092717), (1018.4109157744374, 332.53712634489625), (780.3226669899223, 539.2979739735542), (907.4224088222575, 539.2979739735542)]
Please click
[(439.3015286933047, 302.1047937934921), (559.2407216900153, 317.32096006919414), (323.83767871885925, 504.39029839988467), (445.5670089244761, 508.86564142214996)]


In [40]:
img2 = array(Image.open('./Images/image2.jpg')) 
img2.shape
#plt.imshow(img1)
#plt.show()

(683, 1024, 3)

#

In [41]:
#get x,y,x',y' arrays from cores
def getArrays(src,dest):
    x=[]
    y=[]
    xdash=[]
    ydash=[]

    for i in range(4):
        x.append(src[i][0])
        y.append(src[i][1])
        xdash.append(dest[i][0])
        ydash.append(dest[i][1])

    return x,y,xdash,ydash

In [42]:
#homography parameters implemntation
def findHomography (src, dest):
    b = dest.flatten().reshape(-1,1)

    x,y,xdash,ydash = getArrays(src,dest)

    M = np.array([
          [x[0],y[0],1,0,0,0,-x[0]*xdash[0], -y[0]*xdash[0]],
          [0,0,0,x[0],y[0],1,-x[0]*ydash[0], -y[0]*ydash[0]],
          [x[1],y[1],1,0,0,0,-x[1]*xdash[1],-y[1]*xdash[1]],
          [0,0,0,x[1],y[1],1,-x[1]*ydash[1],-y[1]*ydash[1]],
          [x[2],y[2],1,0,0,0,-x[2]*xdash[2],-y[2]*xdash[2]],
          [0,0,0,x[2],y[2],1,-x[2]*ydash[2],-y[2]*ydash[2]],
          [x[3],y[3],1,0,0,0,-x[3]*xdash[3],-y[3]*xdash[3]],
          [0,0,0,x[3],y[3],1,-x[3]*ydash[3],-y[3]*ydash[3]]
      ])
    a = np.dot(np.linalg.inv(M),b)
    a = np.append(a,1)
    return a.reshape(3,3)

In [43]:
#test to check homography function
pts_src_test = np.array([[141, 131], [480, 159], [493, 630],[64, 601]])
pts_dst_test = np.array([[318, 256],[534, 372],[316, 670],[73, 473]])
h_test=findHomography(pts_src_test,pts_dst_test)
print("calculated homography", h_test)
h_test, status_tst = cv2.findHomography(pts_src_test, pts_dst_test)
print("actual homogrpahy", h_test)


calculated homography [[ 4.34043935e-01 -4.19622184e-01  2.91709494e+02]
 [ 1.46491654e-01  4.41418278e-01  1.61369294e+02]
 [-3.62463336e-04 -9.14274844e-05  1.00000000e+00]]
actual homogrpahy [[ 4.34043935e-01 -4.19622184e-01  2.91709494e+02]
 [ 1.46491654e-01  4.41418278e-01  1.61369294e+02]
 [-3.62463336e-04 -9.14274844e-05  1.00000000e+00]]


In [44]:
h=findHomography(np.array(pts_dst),np.array(pts_src))
print("calculated homography", h)
h, status = cv2.findHomography(np.array(pts_dst),np.array(pts_src))
print("actual homogrpahy", h)

calculated homography [[ 8.65061049e-01  9.79460461e-02  4.25890618e+02]
 [-1.21117878e-01  9.96456025e-01  5.87159752e+01]
 [-1.58060509e-04  3.82428723e-05  1.00000000e+00]]
actual homogrpahy [[ 8.65061592e-01  9.79463747e-02  4.25890533e+02]
 [-1.21117639e-01  9.96456507e-01  5.87158212e+01]
 [-1.58060066e-04  3.82431689e-05  1.00000000e+00]]


# Warping

# Forward

In [45]:
def correct(A):
    if A[2]!=1:
        A[0] = (A[0]/A[2])
        A[1] = (A[1]/A[2])
        A[2] = (A[2]/A[2])
    return A
def round_arr(A):
    A[0]= round(A[0])
    A[1]= round(A[1])
    A[2]= round(A[2])
    return A  

In [46]:
def Get_max_min_Axis (A,B,C,D):
    
    min_x = min(A[0],B[0],C[0],D[0])
    max_x = max(A[0],B[0],C[0],D[0])
    min_y = min(A[1],B[1],C[1],D[1])
    max_y = max(A[1],B[1],C[1],D[1])
   
    return min_x , max_x , min_y , max_y

In [47]:
def Get_new_boundaries (h,N,M):
    A= [0,0,1]
    A_new  = np.dot(h,A)
    A_new = correct(A_new)
    A_new = round_arr(A_new)
    print("old value of A = ",A,"new value = ",A_new)
    B=[0,682,1]
    B_new = np.matmul(h,B )
    B_new = correct(B_new)
    B_new = round_arr(B_new)
    C=[1023,0,1]
    print("old value of B = ",B,"new value = ",B_new)
    C_new = np.matmul(h,C )
    C_new = correct(C_new)
    C_new = round_arr(C_new)
    D=[1023,682,1]
    print("old value of C= ",C,"new value = ",C_new)
    D_new = np.matmul(h,D )
    D_new = correct(D_new)
    D_new = round_arr(D_new)
    print("old value of D = ",D,"new value = ",D_new)
    #print(A_new,B_new,C_new,D_new)
    return A_new,B_new,C_new,D_new

In [48]:
A,B,C,D = Get_new_boundaries(h,img2.shape[0],img2.shape[1])

old value of A =  [0, 0, 1] new value =  [426.  59.   1.]
old value of B =  [0, 682, 1] new value =  [480. 720.   1.]
old value of C=  [1023, 0, 1] new value =  [ 1.564e+03 -7.800e+01  1.000e+00]
old value of D =  [1023, 682, 1] new value =  [1.594e+03 7.110e+02 1.000e+00]


In [13]:
a= np.dot(h,[0,0,1])
print(a)
b= np.dot(h,[0,682,1])
print(b/8.35748000e-01)

[-466.18366395 -147.59087712    1.        ]
[-641.14371397  694.99759794    1.14743001]


In [49]:
min_x , max_x , min_y, max_y = Get_max_min_Axis (A,B,C,D)
print(min_x , max_x , min_y, max_y)

426.0 1594.0 -78.0 720.0


In [50]:
new_w = int(round(abs(max_x - min_x)))
new_h = int(round(abs(max_y - min_y)))

print(new_w,new_h)

1168 798


In [118]:
print(abs(min_x)+int((min_x+new_w)))

1126.0


In [51]:
h_inv = np.linalg.inv(h)
#h_inv

In [52]:
n=np.zeros((new_w,new_h))
print(n.shape)
print(img2.shape)

(1168, 798)
(683, 1024, 3)


In [53]:
gray= cv2.cvtColor(img2,cv2.COLOR_BGR2GRAY)

In [54]:
co_ord = np.matmul(h_inv,D)
print(correct(co_ord))

[1.02315838e+03 6.82190139e+02 1.00000000e+00]


In [55]:
print(img2.shape)
gray.shape

(683, 1024, 3)


(683, 1024)

In [56]:
x = np.arange(0, img2.shape[0], 1)
y = np.arange(0, img2.shape[1], 1)
interp_spline = RectBivariateSpline(x.reshape(-1,1), y.reshape(-1,1), gray)
x=x.reshape(-1,1)
x.shape

(683, 1)

In [34]:
new_img = np.zeros((new_w,new_h))
new_img [int(round(xi)),int(round(yi))]=1


In [58]:
new_img = np.zeros((new_h,new_w))
l=[]

for i in range (int(min_y),int(min_y+new_h),1):
    for j in range (int(min_x),int((min_x+new_w)),1):
        point = [i,j,1]
        co_ord = np.matmul(h_inv,point)
        co_ord = correct(co_ord)
        xi=co_ord[0]
        yi=co_ord[1]
        #interp_spline.ev(xi, yi)
        #print(xi,yi)
        if (xi>=0 and yi>=0):
            intensity=int(interp_spline.ev(xi, yi))
           # print(intensity.shape)
            l.append(intensity)
            new_img[(int(abs(min_y)+i)),(int(abs(min_x)+j))]=intensity
        else:
            l.append(0)

            
            
        


IndexError: index 1168 is out of bounds for axis 1 with size 1168

In [1]:
#plt.plot (np.array(l).reshape(new_h,new_w))
plt.figure()
plt.imshow(new_img)
plt.show()
plt.close()



NameError: name 'plt' is not defined

In [ ]:
plt.imshow(np.array(l).reshape(new_h,new_w))
plt.show()
plt.close()